# PREPARATION

## Import Required Library

In [1]:
import math
from itertools import permutations
from itertools import product
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json

## Global Helper Function

In [2]:
def create_criteria_decision_form() -> None: 
    global total_criteria
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias['type'] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            print(criterias)
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [3]:
def create_matrix_form() -> None:
    global total_criteria, total_alternative, criteria_labels, alternative_labels

    inputs = []
    for i in range(total_alternative):
        rows = []
        for c in range(total_criteria):
            rows.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
        inputs.append(rows)

    header_row = []
    for i in range(len(criteria_labels) + 1):
        header_row.append(widgets.Label(
            value='' if i == 0 else criteria_labels[i - 1],
            layout=widgets.Layout(width="100px")
        ))
    
    input_rows = []
    for r in range(total_alternative):
        row_widgets = [widgets.Label(value=alternative_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        
        decision_matrix.loc[:, :] = get_matrix()

        with output:
            print("Decision matrix:")
            print(decision_matrix)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

In [4]:
def create_matrix_weight() -> None:
    global total_criteria, criteria_labels

    inputs = []
    for i in range(total_alternative):
        inputs.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
    
    input_rows = []
    for r in range(total_criteria):
        row_widgets = [widgets.Label(value=criteria_labels[r], layout=widgets.Layout(width="100px"))] + [inputs[r]]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return [row.value for row in inputs]

    output = widgets.Output()
    
    def update_matrix(_):
        global weights
        
        weights['weight'] = get_matrix()

        with output:
            print("Weight Criteria:")
            print(weights)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Starter Matrix-Like Data Structure of Criterias and Alternatives

In [5]:
total_criteria = 5
total_alternative = 5

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

Define criterias

In [6]:
criterias = pd.DataFrame(None, index=criteria_labels, columns=['type'])
print(criterias)

   type
C1  NaN
C2  NaN
C3  NaN
C4  NaN
C5  NaN


Decide which criteria is benefit or cost

In [7]:
create_criteria_decision_form()

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Create matrix for decision

In [8]:
decision_matrix = pd.DataFrame(0, index=alternative_labels, columns=criteria_labels)

print(decision_matrix)

    C1  C2  C3  C4  C5
A1   0   0   0   0   0
A2   0   0   0   0   0
A3   0   0   0   0   0
A4   0   0   0   0   0
A5   0   0   0   0   0


In [9]:
create_matrix_form()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Normalization

Formula:

**Benefit criteria:**
$$
r_{ij} = \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

**Cost criteria:**
$$
r_{ij} = 1 - \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

In [10]:
matrix_r = decision_matrix.copy(deep=True).astype(float)

for i in range(total_criteria):
    sum_col = (decision_matrix.iloc[:, i] ** 2).sum()

    for j, row in enumerate(decision_matrix.itertuples(index=False)):
        if criterias.iloc[i]['type'] == 'Benefit':
            matrix_r.iloc[j, i] = decision_matrix.iloc[j, i] / math.sqrt(sum_col) # benefit criteria normalization
        else:
            matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization

print(matrix_r)

          C1        C2        C3        C4        C5
A1  0.708270  0.660724  0.730320  0.742391  0.229170
A2  0.367918  0.466393  0.640427  0.117220  0.392862
A3  0.756892  0.466393  0.685373  0.312586  0.458339
A4  0.659648  0.349795  0.325800  0.195366  0.392862
A5  0.416540  0.077732  0.505587  0.547025  0.654771


# Define Weights

In [11]:
weights = pd.DataFrame(0, index=criteria_labels, columns=['weight'])
print(weights)

    weight
C1       0
C2       0
C3       0
C4       0
C5       0


In [12]:
create_matrix_weight()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Calculate Weighted Matrix

Formula:

$$
v_{ij} = r_{ij} \cdot w_{ij}
$$

In [13]:
weighted_matrix = matrix_r.mul(weights['weight'], axis=1)
print(weighted_matrix)

           C1        C2         C3         C4        C5
A1  11.332320  5.285790  10.954801  15.590220  2.750037
A2   5.886694  3.731146   9.606401   2.461614  4.714348
A3  12.110267  3.731146  10.280601   6.564303  5.500073
A4  10.554374  2.798359   4.887002   4.102689  4.714348
A5   6.664641  0.621858   7.583802  11.487531  7.857247


# Get The Concordance and Discordance Set

Calculate the permutation of alternative to see how many group should concordance and discordance have

In [14]:
permutation_alternative = math.perm(total_alternative, 2)
print(permutation_alternative)

20


## Concordance

In [15]:
concordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] >= weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            concordances_data.append(i)
    else:
        concordances_data.append([a, b, None])

concordances = pd.DataFrame(concordances_data, columns=['First', 'Second', 'Selected Criteria'])
print(concordances)

   First Second Selected Criteria
0     A1     A2                C1
1     A1     A2                C2
2     A1     A2                C3
3     A1     A2                C4
4     A1     A3                C2
5     A1     A3                C3
6     A1     A3                C4
7     A1     A4                C1
8     A1     A4                C2
9     A1     A4                C3
10    A1     A4                C4
11    A1     A5                C1
12    A1     A5                C2
13    A1     A5                C3
14    A1     A5                C4
15    A2     A1                C5
16    A2     A3                C2
17    A2     A4                C2
18    A2     A4                C3
19    A2     A4                C5
20    A2     A5                C2
21    A2     A5                C3
22    A3     A1                C1
23    A3     A1                C5
24    A3     A2                C1
25    A3     A2                C2
26    A3     A2                C3
27    A3     A2                C4
28    A3     A

## Discordance

In [16]:
discordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] < weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            discordances_data.append(i)
    else:
        discordances_data.append([a, b, None])

discordances = pd.DataFrame(discordances_data, columns=['First', 'Second', 'Selected Criteria'])
print(discordances)

   First Second Selected Criteria
0     A1     A2                C5
1     A1     A3                C1
2     A1     A3                C5
3     A1     A4                C5
4     A1     A5                C5
5     A2     A1                C1
6     A2     A1                C2
7     A2     A1                C3
8     A2     A1                C4
9     A2     A3                C1
10    A2     A3                C3
11    A2     A3                C4
12    A2     A3                C5
13    A2     A4                C1
14    A2     A4                C4
15    A2     A5                C1
16    A2     A5                C4
17    A2     A5                C5
18    A3     A1                C2
19    A3     A1                C3
20    A3     A1                C4
21    A3     A2              None
22    A3     A4              None
23    A3     A5                C4
24    A3     A5                C5
25    A4     A1                C1
26    A4     A1                C2
27    A4     A1                C3
28    A4     A

# Define The Matrix of Concordance and Discordance

## Concordance

Formula

$$
c_{kl} = \Sigma_{j \in c_{kl}} W_j
$$

In [37]:
sum_weights_concordance = {f"{a[-1]}-{b[-1]}" : 0 if a != b else None for a, b in product(alternative_labels, repeat=2)}

for val in concordances.itertuples(index=False):
    sum_weights_concordance[val[0][-1]+'-'+val[1][-1]] += weights.loc[val[2]].weight if val[2] != None else 0

# print(json.dumps(sum_weights_concordance, indent=4))

Convert it to matrix-like data structure (Pandas DataFrame)

In [38]:
last_dict_item_concordance = list(sum_weights_concordance.keys())[-1]

concordance_matrix = pd.DataFrame(np.nan, index=range(int(last_dict_item_concordance[0])), columns=range(int(last_dict_item_concordance[-1])))

for key, value in sum_weights_concordance.items():
    row, col = map(int, key.split('-'))
    concordance_matrix.at[row-1, col-1] = value
    
print(concordance_matrix)

      0     1     2     3     4
0   NaN  60.0  44.0  60.0  60.0
1  12.0   NaN   8.0  35.0  23.0
2  28.0  72.0   NaN  72.0  39.0
3  12.0  49.0   0.0   NaN  24.0
4  12.0  49.0  33.0  48.0   NaN


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


## Discordance

Formula

$$
d_{kl} = \frac{max(|v_{kj} - v_{vj}|)_{j \in D_{kl}}}{max(|v_{kj} - v_{lj}|)_{\forall_{j}}}
$$

In [39]:
discordances_group = {f"{a[-1]}-{b[-1]}" : [] if a != b else None for a, b in product(alternative_labels, repeat=2)}
# print(json.dumps(discordances_group, indent=4))

In [40]:
for val in discordances.itertuples(index=False):
    a = val[0][-1]
    b = val[1][-1]
    discordances_group[a+'-'+b].append(val[2])

# print(json.dumps(discordances_group, indent=4))

In [41]:
discordance_matrix_data = {f"{a[-1]}-{b[-1]}" : 0 if a != b else None for a, b in product(alternative_labels, repeat=2)}

for key, val in discordances_group.items():
    if val == None: continue
        
    a1 = f"A{key[0]}"
    a2 = f"A{key[-1]}"

    difference_criteria = []
    difference_criteria_discordance = []
    for criteria in weighted_matrix.columns:
        if criteria in val:
            difference_criteria_discordance.append(abs(weighted_matrix.loc[a1][criteria] - weighted_matrix.loc[a2][criteria]))
        else:
            difference_criteria.append(abs(weighted_matrix.loc[a1][criteria] - weighted_matrix.loc[a2][criteria]))

    discordance_matrix_data[key] = max(difference_criteria_discordance, default=float('-inf')) / max(difference_criteria, default=float('-inf'))

# print(json.dumps(discordance_matrix_data, indent=4))

<ipython-input-41-3e84a9837aea>:17: RuntimeWarning: divide by zero encountered in scalar divide
  discordance_matrix_data[key] = max(difference_criteria_discordance, default=float('-inf')) / max(difference_criteria, default=float('-inf'))


Convert data structure to matrix-like data type (Pandas DataFrame)

In [42]:
last_dict_item_discordance = list(discordance_matrix_data.keys())[-1]

discordance_matrix = pd.DataFrame(np.nan, index=range(int(last_dict_item_discordance[0])), columns=range(int(last_dict_item_discordance[-1])))

for key, value in discordance_matrix_data.items():
    row, col = map(int, key.split('-'))
    discordance_matrix.at[row-1, col-1] = value
    
print(discordance_matrix)

          0         1         2         3         4
0       NaN  0.149621  0.304682  0.170995  1.094165
1  6.683565       NaN       inf  0.989041  2.902888
2  3.282108      -inf       NaN      -inf  0.904070
3  5.848120  1.011080 -0.000000       NaN  1.898547
4  0.913939  0.344484  1.106109  0.526719       NaN


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


# Calculate the Dominant of The Matrix

Threshold formula:

$$
\underline{c} = \frac{\Sigma_{k=1}^m \Sigma_{l=1}^m c_{kl}}{m(m-1)}
$$

In [43]:
def calculate_threshold(matrix: pd.DataFrame):
    return matrix.sum().sum() / (matrix.shape[0] * (matrix.shape[0] - 1))

## Concordance

Compare each value in matrix with **threshold (c)**

So, the value of matrix F is:
$$
f_{kl} = 
\begin{cases}
    1, \text{if } c_{kl} \geq \underline{c} \\
    0, \text{if } c_{kl} \lt \underline{c}
\end{cases}
$$

In [44]:
concordance_threshold = calculate_threshold(concordance_matrix)
print(concordance_threshold)

37.0


In [45]:
dominant_concordance = concordance_matrix.applymap(lambda x: 1 if x >= concordance_threshold else 0)
print(dominant_concordance)

   0  1  2  3  4
0  0  1  1  1  1
1  0  0  0  0  0
2  0  1  0  1  1
3  0  1  0  0  0
4  0  1  0  1  0


<ipython-input-45-4711211d3c29>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dominant_concordance = concordance_matrix.applymap(lambda x: 1 if x >= concordance_threshold else 0)


## Discordance

Compare each value in matrix with **threshold (c)**.

So, the value of matrix F is:
$$
g_{kl} = 
\begin{cases}
    1, \text{if } d_{kl} \geq \underline{d} \\
    0, \text{if } d_{kl} \lt \underline{d}
\end{cases}
$$

In [46]:
discordance_threshold = calculate_threshold(discordance_matrix)
print(discordance_threshold)

nan


/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [47]:
dominant_discordance = discordance_matrix.applymap(lambda x: 1 if x >= discordance_threshold else 0)
print(dominant_discordance)

   0  1  2  3  4
0  0  0  0  0  0
1  0  0  0  0  0
2  0  0  0  0  0
3  0  0  0  0  0
4  0  0  0  0  0


<ipython-input-47-a6b629297ef0>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dominant_discordance = discordance_matrix.applymap(lambda x: 1 if x >= discordance_threshold else 0)


# Calculate Aggregate Dominant Matrix

Formula:

$$
e_{kl} = f_{kl} \times g_{kl}
$$

In [ ]:
print('Calculation should be here')

# Eliminate Less Favourable Alternatives

In [ ]:
print('Calculation should be here')

# Rangking Alternative

In [ ]:
print('Calculation should be here')

---

# Conclusion